In [169]:
import pandas as pd
import datetime as dt
import json
import os

def stringtodate(date, time, timezone):
    string = date+' '+time+timezone[4:10].replace(':','')
    return dt.datetime.strptime(string,"%d/%m/%Y %H:%M:%S%z")

def formatstring(date, time, timezone):
    date = stringtodate(date,time,timezone)
    return dt.datetime.strftime(date,"%Y-%m-%dT%H:%M:%S"+timezone[4:10])

In [4]:
data = pd.read_csv('data.csv')

In [3]:
base_string = \
'''+++
  host= "{host}"
  date = "{start_time}"
  expiryDate = "{end_time}"
  title = "{title}"
  speaker = "{speaker}"
  speaker_institution = "{speaker_inst}"
  talk_site = "{talk_site}"
  categories = [{categories}]

  publishDate = "2000-02-07T16:00:00-07:00"
+++

{abstract}'''

In [21]:

    row = row[1].to_dict()

In [104]:
parser = {
    'host': lambda x:x['Host'],
    'title': lambda x: x['Title'],
    'start_time' : lambda x : formatstring(x['Date'],x['Start_Time'], x['Timezone']),
    'end_time' : lambda x : formatstring(x['Date'],x['End_Time'], x['Timezone']),
    'speaker' : lambda x: x['Speaker'],
    'speaker_inst': lambda x: x['Speaker_inst'],
    'talk_site': lambda x: x['Site'],
    'categories': lambda x : '' if x['arXiv']=='' else '"'+'","'.join(x['arXiv'].upper().replace(' ','').replace('MATH.','').split(','))+'"',
    'abstract': lambda x: x['Abstract'],
    'name': lambda x: x['Host']+'_'+x['Speaker'].replace(' ','_')+'_'+x['Date'].replace('/','')+'.md'
}

In [105]:
'math.NT, math.DT, math.BD'

'math.NT, math.DT, math.BD'

In [108]:
for row in data.iterrows():
    mydict = { prop:func(row[1]) for prop, func in parser.items()}
    with open(mydict['name'], 'w') as f:
        f.write(base_string.format(**mydict))

In [96]:
base_string.format(**mydict)

'\n+++\n  host= "UCLA"\n  date = "2020-04-15T09:00:00-07:00"\n  expiryDate = "2020-04-15T10:00:00-07:00"\n  title = "The life of the pandas"\n  speaker = "Irina"\n  speaker_institution = "NYU CDS"\n  talk_site = "https://www.worldwildlife.org/"\n  categories = ["NT","GR"]\n\n  publishDate = "2000-02-07T16:00:00-07:00"\n+++\n\nPandas are beautiful $a^b$ we are testing latex.\n'

In [91]:
mydict.keys()

dict_keys(['host', 'title', 'start_time', 'end_time', 'speaker', 'speaker_inst', 'talk_site', 'categories'])

In [64]:
parser.items()

dict_items([('host', <function <lambda> at 0x10f6cca60>), ('title', <function <lambda> at 0x10f6cc730>), ('start_time', <function <lambda> at 0x10f6cc268>), ('end_time', <function <lambda> at 0x10f6ccbf8>), ('speaker', <function <lambda> at 0x10f6ccc80>), ('speaker_inst', <function <lambda> at 0x10f6ccd08>), ('talk_site', <function <lambda> at 0x10f6ccd90>), ('categories', <function <lambda> at 0x10f6cce18>)])

In [97]:
data

,Timestamp,Email,Title,Speaker,Speaker_inst,Abstract,Host,Site,In_Charge,arXiv,Date,Start_Time,End_Time,Timezone
0,6/04/2020 19:55:10,jaumededios@gmail.com,The life of the pandas,Irina,NYU CDS,Pandas are beautiful $a^b$ we are testing latex.,UCLA,https://www.worldwildlife.org/,Yes,"math.NT, math.GR",15/04/2020,9:00:00,10:00:00,"UTC -07:00 (Los Angeles, PDT)"


In [109]:
with open('old_talk/BerkeleyPawloski.md') as f:
    ot = f.read()

In [112]:
ot.split('+++')[-1].lstrip()

'Huang, McKinnon, and Satriano conjectured that if a real vector $(v_1,...,v_n)$ has distinct coordinates and $n\\ge3$, then a hyperplane through the origin other than $x_1+...+x_n=0$ contains at most $2(n−2)!\\lfloor n/2\\rfloor$ of the vectors obtained by permuting the coordinates of $v$. I will discuss a proof of this conjecture.'

In [ ]:
inverse_parser = {
    'host': lambda x:x['Host'],
    'title': lambda x: x['Title'],
    'start_time' : lambda x : formatstring(x['Date'],x['Start_Time'], x['Timezone']),
    'end_time' : lambda x : formatstring(x['Date'],x['End_Time'], x['Timezone']),
    'speaker' : lambda x: x['Speaker'],
    'speaker_inst': lambda x: x['Speaker_inst'],
    'talk_site': lambda x: x['Site'],
    'categories': lambda x : '' if x['arXiv']=='' else '"'+'","'.join(x['arXiv'].upper().replace(' ','').replace('MATH.','').split(','))+'"',
    'abstract': lambda x: x['Abstract'],
    'name': lambda x: x['Host']+'_'+x['Speaker'].replace(' ','_')+'_'+x['Date'].replace('/','')+'.md'
}

In [153]:
def clean_text(ot):
    base = list(map(lambda x: x.split('='), ot.split('+++')[1].lstrip().rstrip().split('\n')))
    clean = { x[0].lstrip().rstrip(): '='.join(x[1:]).lstrip().rstrip()[1:-1] for x in base}
    clean['abstract'] = ot.split('+++')[-1].lstrip().rstrip()
    clean['categories'] = ', '.join(map(lambda x: 'Math.'+x, clean['categories'].replace('"','').split(',')))
    return clean

In [154]:
ct = clean_text(ot)

In [166]:
inverse_parser = {
    'Timestamp': lambda x:'-1',
    'Email': lambda x:'jaumededios@gmail.com',
    'Title': lambda x: x['title'],
    'Speaker' : lambda x: x['speaker'],
    'Speaker_inst': lambda x: x['speaker_institution'],
    'Host': lambda x: x['host'],
    'Abstract': lambda x: x['abstract'],
    'Site': lambda x: x['talk_site'],
    'arXiv': lambda x : x['categories'],
    'Date': lambda x: x['date'][8:10]+'/'+x['date'][5:7]+'/'+x['date'][:4],
    'Start_Time' : lambda x : x['date'][11:19],
    'End_Time' : lambda x : x['expiryDate'][11:19],
    'Timezone': lambda x : 'UTC '+x['date'][-6:],
    'In_Charge': lambda x:  False,
}

In [167]:
[inverse_parser[prop](ct) for prop in data.columns]

['-1',
 'jaumededios@gmail.com',
 'The fraction of an Sn-orbit on a hyperplane',
 'Brendan Pawlowski',
 'USC',
 'Huang, McKinnon, and Satriano conjectured that if a real vector $(v_1,...,v_n)$ has distinct coordinates and $n\\ge3$, then a hyperplane through the origin other than $x_1+...+x_n=0$ contains at most $2(n−2)!\\lfloor n/2\\rfloor$ of the vectors obtained by permuting the coordinates of $v$. I will discuss a proof of this conjecture.',
 'UC Berkeley',
 'https://events.berkeley.edu/index.php/calendar/sn/math.html?event_ID=132639&date=2020-04-06&filter=Secondary%20Event%20Type&filtersel=',
 False,
 'Math.CO',
 '06/04/2020',
 '12:10:00',
 '13:00:00',
 'UTC -07:00']

In [175]:
old_files = []
for filename in os.listdir('old_talk/'):
    if filename[0]!='.':
        with open('old_talk/'+filename) as f:
            ot = f.read()
        print(filename)
        ct = clean_text(ot)
        old_files.append([inverse_parser[prop](ct) for prop in data.columns])

UCLAShimizu.md
BerkeleyPawloski.md
PrincetonLi.md
SeymourStanford.md
CaltechGhomi.md
VANTAGEFite.md
VANTAGEZywina.md
UCLAHunyh.md
ColumbiaBray.md
VANTAGESutherland.md
VANTAGEBucur.md
BerkeleyYing.md
OSUIncerti.md
NCCChudnovsky.md
CaltechTAO.md
PrincetonBeryland.md
UCLA_Irina_15042020.md
SIUDobrinen.md
ColumbiaMihatsch.md


In [187]:
old_talk_data = pd.DataFrame(old_files, columns=data.columns)
old_talk_data.to_csv('old_talk_data.csv')